In [15]:
"""deprecated OOD eval code"""

from trl import PPOConfig, PPOTrainer
import utils
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoModel,
    Trainer,
    TrainingArguments,
    BertModel,
    pipeline,
    AutoModelForSequenceClassification,
    BitsAndBytesConfig,
)
from transformers import AutoModelForCausalLM, AutoTokenizer
import yaml
import getpass
import wandb
from typing import Dict, Any
import torch as t
# from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
from tqdm import tqdm
import trl
import importlib

device = t.device("cuda" if t.cuda.is_available() else "cpu")

## eval
- load models and reward models
- load in-distribution and out-of-distribution datasets
- generate outputs from models
- feed generated outputs to the reward model and store them 

In [16]:
# copied from train rlhf, might need to change
def setup_logging(hps: Dict[str, Any], log_wandb):
    # Choose logging and checkpoint saving directory
    logdir = utils.choose_log_dir(
        f"{utils.run_dir}/{hps['dataset_name']}/training/{hps['training_algorithm']}",
        debug=hps["debug"],
    )

    # Add a couple of keys to the hps object and save it as a yaml file
    hps["logdir"] = logdir

    hps["training_kwargs"]["run_name"] = "/".join(logdir.split("/")[-2:])
    hps["user"] = getpass.getuser()
    hps["tags"] += [
        hps["dataset"]["name"],
        "training",
        hps["training_algorithm"],
    ]
    with open(f"{logdir}/hps.yaml", "w") as f:
        yaml.dump(hps, f)

    # If not in debug mode, setup wandb logging
    if not hps["debug"] or log_wandb:
        wandb.init(
            project="dpo_rlhf_generalization",
            dir=logdir,
            name=hps["training_kwargs"]["run_name"],
            config=utils.wandb_configify(hps),
            tags=hps["tags"],
            save_code=True,
            settings=wandb.Settings(code_dir="."),
        )

    print(f"Hyperparameters:\n{hps}\n")
    return logdir

In [17]:
def load_model(checkpoint_path: str):
    """
    load model from cehckpoint and wrap with peft if neded
"""

    # Load tokenizer
    tokenizer = transformers.AutoTokenizer.from_pretrained(checkpoint_path)
    tokenizer.pad_token = tokenizer.eos_token

    # Load model from the specific checkpoint
    model = transformers.AutoModelForCausalLM.from_pretrained(
                checkpoint_path, torch_dtype=t.bfloat16
                    ).to(t.device("cuda:0"))

    return tokenizer, model

In [18]:
args = 'hyperparams/rlhf.yaml'
with open(
    args
) as f:
    hps = yaml.load(f, Loader=yaml.FullLoader)

In [19]:
# Load the reward model
# reward_model = AutoModelForSequenceClassification.from_pretrained("reward_model_path").to(device)
reward_model = AutoModelForSequenceClassification.from_pretrained(hps["rm_path"]).to(device)

# Load the models
model_paths = ["./dpo_dahoas_instruct/checkpoint-6000", "./dpo_anthropic_instruct/checkpoint-8000"]
tokenizers, models = [load_model(path) for path in model_paths]


OSError: Incorrect path_or_model_id: './dpo_dahoas_instruct/checkpoint-6000'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:


# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("tokenizer_path")

# Load the datasets
in_distribution_dataset = hps["in-dataset"] #...  # Load your in-distribution dataset
out_of_distribution_dataset = ...  # Load your out-of-distribution dataset

# Function to generate outputs from a model
def generate_outputs(model, dataset):
    outputs = []
    for input in dataset:
        input_tensor = tokenizer.encode(input, return_tensors="pt").to(device)
        output_tensor = model.generate(input_tensor)
        output = tokenizer.decode(output_tensor[0])
        outputs.append(output)
    return outputs

# Function to get rewards from the reward model
def get_rewards(reward_model, outputs):
    rewards = []
    for output in outputs:
        input_tensor = tokenizer.encode(output, return_tensors="pt").to(device)
        reward = reward_model(input_tensor).logits.item()
        rewards.append(reward)
    return rewards

In [ ]:

# Evaluate the models
for model in models:
    for dataset, dataset_name in [(in_distribution_dataset, "in-distribution"), (out_of_distribution_dataset, "out-of-distribution")]:
        outputs = generate_outputs(model, dataset)
        rewards = get_rewards(reward_model, outputs)
        print(f"Model: {model}, Dataset: {dataset_name}, Average Reward: {sum(rewards) / len(rewards)}")

In [ ]:
# ALPACA 

import datasets

eval_set = datasets.load_dataset("tatsu-lab/alpaca_eval", "alpaca_eval")["eval"]
for example in eval_set:
    # generate here is a placeholder for your models generations
    example["output"] = generate(example["instruction"])